<a href="https://colab.research.google.com/github/sahana0007/Spark/blob/main/Pyspark_rdd_to_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 46.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=5085635e57584311eec8268efa9e404cdeeb6887162119aeda5aae24ca20c55e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Data pipeline RDD").\
        getOrCreate()
#conf = SparkConf().setAppName("Data pipeline RDD").setMaster("local")
#sc = SparkContext (conf = conf)

In [4]:
df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("/content/drive/MyDrive/Colab Notebooks/supermarket_sales.csv")
    #.csv("s3a://sparkbuckets")\


In [6]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.window import * 
from pyspark.sql.types import * 
from pyspark.sql.functions import split
from pyspark.sql.functions import udf


In [10]:
df.show()

+-----------+---------+------+--------------------+----------+--------+-------+--------+---------+
| Invoice ID|     City|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date|
+-----------+---------+------+--------------------+----------+--------+-------+--------+---------+
|750-67-8428|   Yangon|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|
|226-31-3081|Naypyitaw|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|
|631-41-3108|   Yangon|  Male|  Home and lifestyle|     46.33|       7|16.2155|340.5255| 3/3/2019|
|123-19-1176|   Yangon|  Male|   Health and beauty|     58.22|       8| 23.288| 489.048|1/27/2019|
|373-73-7910|   Yangon|  Male|   Sports and travel|     86.31|       7|30.2085|634.3785| 2/8/2019|
|699-14-3026|Naypyitaw|  Male|Electronic access...|     85.39|       7|29.8865|627.6165|3/25/2019|
|355-53-5943|   Yangon|Female|Electronic access...|     68.84|       6| 20.652| 433.692|2/25/2019|
|315-22-56

Covert df to rdd to create calculation field on quantity. Remember that RDD is schemaless

In [16]:
rdd = df.rdd.map(lambda x: (x[1], x[5], x[5] *2))
print(rdd.collect())

[('Yangon', 7, 14), ('Naypyitaw', 5, 10), ('Yangon', 7, 14), ('Yangon', 8, 16), ('Yangon', 7, 14), ('Naypyitaw', 7, 14), ('Yangon', 6, 12), ('Naypyitaw', 10, 20), ('Yangon', 2, 4), ('Mandalay', 3, 6), ('Mandalay', 4, 8), ('Mandalay', 4, 8), ('Yangon', 5, 10), ('Yangon', 10, 20), ('Yangon', 10, 20), ('Mandalay', 6, 12), ('Yangon', 7, 14), ('Yangon', 6, 12), ('Yangon', 3, 6), ('Mandalay', 2, 4), ('Naypyitaw', 5, 10), ('Mandalay', 3, 6), ('Mandalay', 2, 4), ('Yangon', 5, 10), ('Yangon', 3, 6), ('Yangon', 8, 16), ('Mandalay', 1, 2), ('Yangon', 2, 4), ('Mandalay', 5, 10), ('Yangon', 9, 18), ('Mandalay', 5, 10), ('Mandalay', 9, 18), ('Mandalay', 8, 16), ('Yangon', 2, 4), ('Naypyitaw', 4, 8), ('Naypyitaw', 1, 2), ('Yangon', 5, 10), ('Yangon', 9, 18), ('Naypyitaw', 8, 16), ('Mandalay', 8, 16), ('Mandalay', 1, 2), ('Naypyitaw', 2, 4), ('Mandalay', 6, 12), ('Naypyitaw', 8, 16), ('Naypyitaw', 2, 4), ('Mandalay', 4, 8), ('Mandalay', 9, 18), ('Mandalay', 9, 18), ('Mandalay', 6, 12), ('Naypyitaw', 1

Converting rdd to df

In [18]:
newSchema = "City","Quantity","Multiquantity"

In [19]:
df1 = spark.createDataFrame(rdd,schema = newSchema)
df1.printSchema()
df1.show(truncate=False)

root
 |-- City: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Multiquantity: long (nullable = true)

+---------+--------+-------------+
|City     |Quantity|Multiquantity|
+---------+--------+-------------+
|Yangon   |7       |14           |
|Naypyitaw|5       |10           |
|Yangon   |7       |14           |
|Yangon   |8       |16           |
|Yangon   |7       |14           |
|Naypyitaw|7       |14           |
|Yangon   |6       |12           |
|Naypyitaw|10      |20           |
|Yangon   |2       |4            |
|Mandalay |3       |6            |
|Mandalay |4       |8            |
|Mandalay |4       |8            |
|Yangon   |5       |10           |
|Yangon   |10      |20           |
|Yangon   |10      |20           |
|Mandalay |6       |12           |
|Yangon   |7       |14           |
|Yangon   |6       |12           |
|Yangon   |3       |6            |
|Mandalay |2       |4            |
+---------+--------+-------------+
only showing top 20 rows

